# Type Coercion

When Pydantic deserializes data, one of the things it does is perform validation. This includes ensuring that the model instance data ends up as the correct type.

Let's look at an example:

In [1]:
from pydantic import BaseModel, ValidationError

In [2]:
class Coordinates(BaseModel):
    x: float
    y: float

Notice how we are insisting that `x` and `y` should be floats.

Let's deseralize some data:

In [3]:
p1 = Coordinates(x=1.1, y=2.2)
p1

Coordinates(x=1.1, y=2.2)

We can see our field definitions:

In [4]:
Coordinates.model_fields

{'x': FieldInfo(annotation=float, required=True),
 'y': FieldInfo(annotation=float, required=True)}

As you can see, the defined type for each field is `float`. 

And indeed, if we check the type of the fields, we get the expected result:

In [5]:
type(p1.x)

float

But what happens if the data we provide for deserializtion is not an exact type match?

Pydantic will attempt to "transform" the data into the correct type - this is called type **coercion**

Let's see this:

In [6]:
p2 = Coordinates(x=0, y="1.2")
p2

Coordinates(x=0.0, y=1.2)

As you can see, Pydantic was able to coerce the **integer** `0`, and the **string** `"1.2"` to a float value:

In [7]:
type(p2.x), type(p2.y)

(float, float)

Pydantic is not always able to perform the type coercion. In fact, we can even choose the level of type coercion that we find acceptable.

By default, the type coercion is termed **lax** - and it attempts a variety of type coercions.

But, as we'll see later, we have the option to change that, to a **strict** mode that is far less forgiving when incorrect data types are provided in the data.

Pydantic docs that describes what type coercions will be attempted in either of these modes, is located here:

[https://docs.pydantic.dev/latest/concepts/conversion_table/](https://docs.pydantic.dev/latest/concepts/conversion_table/)

If you look at that conversion table, you'll notice, for example, that in lax mode, and dealing with Python data types, input data that is float, int, or Decimal will be coerced to a float. However, strings wil be coerced to floats only under certain conditions.

In strict mode, notice that string to float conversion is not supported (so it will raise a validation error).

Use this table when considering type coercion because things are not always "obvious".

For example, with this model:

In [8]:
class Model(BaseModel):
    field: str

We know that all objects in Python have a `str()` representation, so we might expect to be able to pass any type for `field` and have Pydantic coerce it to a string.

But that is not the case:

In [9]:
try:
    Model(field=100)
except ValidationError as ex:
    print(ex)

1 validation error for Model
field
  Input should be a valid string [type=string_type, input_value=100, input_type=int]
    For further information visit https://errors.pydantic.dev/2.5/v/string_type


And this is probably a good thing as allowing this could lead to unintended problems where we deserialize and object to a string when we never intended for that to happen (our source data maybe changed on us - and auto coercing to a string would hide a potential bug).

Here's what I mean:

We are querying a REST API and getting some JSON back from that API, which we model this way:

In [10]:
class Contact(BaseModel):
    email: str

In [11]:
initial_json_data = '''
{
    "email": "inewton@principia.com"
}
'''

This deserializes just fine:

In [12]:
Contact.model_validate_json(initial_json_data)

Contact(email='inewton@principia.com')

But now, suppose that API changes it's response model, and we are unaware of the change.

The response data now looks like this:

In [13]:
new_json_data = '''
{
    "email": {
        "personal": "inewton@principia.com",
        "work": "isaac.newton@themint.com"
    }
}
'''

Tring to deserialize this data will not work, and we'll immediately know something is wrong with our app (and we can then go in and fix it):

In [14]:
try:
    Contact.model_validate_json(new_json_data)
except ValidationError as ex:
    print(ex)

1 validation error for Contact
email
  Input should be a valid string [type=string_type, input_value={'personal': 'inewton@pri...aac.newton@themint.com'}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.5/v/string_type


What if Pydantic had instead just decided to deserialize that `email` complex object (a nested dictionary basically) into it's string representation?

We can mimic the behavior this way:

In [15]:
new_data = {
    "email": {
        "personal": "inewton@principia.com",
        "work": "isaac.newton@themint.com"
    }
}

In [16]:
Contact(email=str(new_data['email']))

Contact(email="{'personal': 'inewton@principia.com', 'work': 'isaac.newton@themint.com'}")

And you can see that we now have a string representation of the email dictionary - no exceptions, and our code is likely to break from that point forward, depending on how we use the `email` field.